In [11]:
using JuMP, GLPK
using DataFrames
using CSV

In [12]:
#Load data on demand 
demand = CSV.File("../data/demand.csv") |> DataFrame

#Load data on generation units offer
offer = CSV.File("../data/cost_generation.csv") |> DataFrame

#Load data on capacity of generation units 
capacity = CSV.File("../data/technical_data_generation.csv") |> DataFrame

#Load data wind
#Load wind production profile
wind_powerprofile = CSV.File("../data/wind_powerprofile.csv") |> DataFrame
wind_inst_capacity = CSV.File("../data/technical_data_generation")

┌ Warning: thread = 1 warning: only found 1 / 7 columns around data row: 25. Filling remaining columns with `missing`
└ @ CSV C:\Users\leonr\.julia\packages\CSV\aoJqo\src\file.jl:586
┌ Warning: thread = 1 warning: only found 1 / 7 columns around data row: 25. Filling remaining columns with `missing`
└ @ CSV C:\Users\leonr\.julia\packages\CSV\aoJqo\src\file.jl:586


ArgumentError: ArgumentError: "../data/technical_data_wind_generation" is not a valid file or doesn't exist

In [54]:
#Initialize data frame for outcome of optimization problem
MarketClearingResults_df = DataFrame(hour=Int[],MarketClearingPrice = Float64[])

#Add production and profit column for each generator 
for i in 1:size(offer, 1)
    MarketClearingResults_df[!, Symbol("ProductionGen$i")] = Float64[]
    #MarketClearingResults_df[!, Symbol("ProfitGen$i")] = Float64[]
end


In [55]:
#Optimize market clearing for each hour 
for hour in 1:24
    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)

    # Define the decision variables for every generator
    unregister(model, :ProductionGen)  # Unregister the existing variable named "x" from the model
    @variable(model, ProductionGen[1:size(offer, 1)])  # Create the new variable
    for generator in 1:size(offer, 1)
        set_lower_bound(ProductionGen[generator], capacity[generator,4])  # Set the lower bound
        set_upper_bound(ProductionGen[generator], capacity[generator,3])  # Set the upper bound
    end

    #Define Objective function -> Minimize cost/offer of generators
    @objective(model, Min, sum(offer[generator,2] * ProductionGen[generator] for generator in 1:size(offer,1)))

    #Add power balance constraint -> equilibrium constraint
    @constraint(model, BalanceConstraint, sum(ProductionGen[generator] for generator in 1:size(offer,1)) == demand[hour,2])

     # Print the model
     print(model)

     # Solve the model
     optimize!(model)
 
     status = termination_status(model)
    if status == MOI.OPTIMAL
        println("Optimal solution found.")
        println("Total generation: ", objective_value(model))
        println("Generation per generator: ", value.(ProductionGen))
        
        # Retrieve dual variable (market clearing price)
        MCP = dual(BalanceConstraint)
        println("Market clearing price: ", MCP)

        new_row = [hour, MCP]
        append!(new_row, [value(ProductionGen[i]) for i in 1:size(offer, 1)]) # Simplifies adding x values

        push!(MarketClearingResults_df, new_row)
    else
        println("No optimal solution found.")
    end
 end

Min 13.32 ProductionGen[1] + 13.32 ProductionGen[2] + 20.7 ProductionGen[3] + 20.93 ProductionGen[4] + 26.11 ProductionGen[5] + 10.52 ProductionGen[6] + 10.52 ProductionGen[7] + 6.02 ProductionGen[8] + 5.47 ProductionGen[9] + 10.52 ProductionGen[11] + 10.89 ProductionGen[12]


Subject to
 BalanceConstraint : ProductionGen[1] + ProductionGen[2] + ProductionGen[3] + ProductionGen[4] + ProductionGen[5] + ProductionGen[6] + ProductionGen[7] + ProductionGen[8] + ProductionGen[9] + ProductionGen[10] + ProductionGen[11] + ProductionGen[12] == 1775.835
 ProductionGen[1] >= 30.4
 ProductionGen[2] >= 30.4
 ProductionGen[3] >= 75
 ProductionGen[4] >= 206.85
 ProductionGen[5] >= 12
 ProductionGen[6] >= 54.25
 ProductionGen[7] >= 54.25
 ProductionGen[8] >= 100
 ProductionGen[9] >= 100
 ProductionGen[10] >= 300
 ProductionGen[11] >= 108.5
 ProductionGen[12] >= 140
 ProductionGen[1] <= 152
 ProductionGen[2] <= 152
 ProductionGen[3] <= 350
 ProductionGen[4] <= 591
 ProductionGen[5] <= 60
 ProductionGen[6] <= 155
 ProductionGen[7] <= 155
 ProductionGen[8] <= 400
 ProductionGen[9] <= 400
 ProductionGen[10] <= 300
 ProductionGen[11] <= 310
 ProductionGen[12] <= 350
Optimal solution found.
Total generation: 15192.8802
Generation per generator: [30.4, 30.4, 75.0, 206.85, 12.0, 

Market clearing price: 6.02
Min 13.32 ProductionGen[1] + 13.32 ProductionGen[2] + 20.7 ProductionGen[3] + 20.93 ProductionGen[4] + 26.11 ProductionGen[5] + 10.52 ProductionGen[6] + 10.52 ProductionGen[7] + 6.02 ProductionGen[8] + 5.47 ProductionGen[9] + 10.52 ProductionGen[11] + 10.89 ProductionGen[12]
Subject to
 BalanceConstraint : ProductionGen[1] + ProductionGen[2] + ProductionGen[3] + ProductionGen[4] + ProductionGen[5] + ProductionGen[6] + ProductionGen[7] + ProductionGen[8] + ProductionGen[9] + ProductionGen[10] + ProductionGen[11] + ProductionGen[12] == 1669.815
 ProductionGen[1] >= 30.4
 ProductionGen[2] >= 30.4
 ProductionGen[3] >= 75
 ProductionGen[4] >= 206.85
 ProductionGen[5] >= 12
 ProductionGen[6] >= 54.25
 ProductionGen[7] >= 54.25
 ProductionGen[8] >= 100
 ProductionGen[9] >= 100
 ProductionGen[10] >= 300
 ProductionGen[11] >= 108.5
 ProductionGen[12] >= 140
 ProductionGen[1] <= 152
 ProductionGen[2] <= 152
 ProductionGen[3] <= 350
 ProductionGen[4] <= 591
 Production

[30.4, 30.4, 75.0, 206.85, 12.0, 54.25, 54.25, 258.16499999999996, 400.0, 300.0, 108.5, 140.0]
Market clearing price: 6.02
Min 13.32 ProductionGen[1] + 13.32 ProductionGen[2] + 20.7 ProductionGen[3] + 20.93 ProductionGen[4] + 26.11 ProductionGen[5] + 10.52 ProductionGen[6] + 10.52 ProductionGen[7] + 6.02 ProductionGen[8] + 5.47 ProductionGen[9] + 10.52 ProductionGen[11] + 10.89 ProductionGen[12]
Subject to
 BalanceConstraint : ProductionGen[1] + ProductionGen[2] + ProductionGen[3] + ProductionGen[4] + ProductionGen[5] + ProductionGen[6] + ProductionGen[7] + ProductionGen[8] + ProductionGen[9] + ProductionGen[10] + ProductionGen[11] + ProductionGen[12] == 1590.3
 ProductionGen[1] >= 30.4
 ProductionGen[2] >= 30.4
 ProductionGen[3] >= 75
 ProductionGen[4] >= 206.85
 ProductionGen[5] >= 12
 ProductionGen[6] >= 54.25
 ProductionGen[7] >= 54.25
 ProductionGen[8] >= 100
 ProductionGen[9] >= 100
 ProductionGen[10] >= 300
 ProductionGen[11] >= 108.5
 ProductionGen[12] >= 140
 ProductionGen[1] 

 ProductionGen[12] <= 350
Optimal solution found.
Total generation: 14075.959499999999
Generation per generator: [30.4, 30.4, 75.0, 206.85, 12.0, 54.25, 54.25, 178.64999999999975, 400.0, 300.0, 108.5, 140.0]
Market clearing price: 6.02
Min 13.32 ProductionGen[1] + 13.32 ProductionGen[2] + 20.7 ProductionGen[3] + 20.93 ProductionGen[4] + 26.11 ProductionGen[5] + 10.52 ProductionGen[6] + 10.52 ProductionGen[7] + 6.02 ProductionGen[8] + 5.47 ProductionGen[9] + 10.52 ProductionGen[11] + 10.89 ProductionGen[12]
Subject to
 BalanceConstraint : ProductionGen[1] + ProductionGen[2] + ProductionGen[3] + ProductionGen[4] + ProductionGen[5] + ProductionGen[6] + ProductionGen[7] + ProductionGen[8] + ProductionGen[9] + ProductionGen[10] + ProductionGen[11] + ProductionGen[12] == 1563.795
 ProductionGen[1] >= 30.4
 ProductionGen[2] >= 30.4
 ProductionGen[3] >= 75
 ProductionGen[4] >= 206.85
 ProductionGen[5] >= 12
 ProductionGen[6] >= 54.25
 ProductionGen[7] >= 54.25
 ProductionGen[8] >= 100
 Product

...

 ProductionGen[6] <= 155
 ProductionGen[7] <= 155
 ProductionGen[8] <= 400
 ProductionGen[9] <= 400
 ProductionGen[10] <= 300
 ProductionGen[11] <= 310
 ProductionGen[12] <= 350
Optimal solution found.
Total generation: 23531.0821
Generation per generator: [30.4, 150.22999999999993, 75.0, 206.85, 12.0, 155.0, 155.0, 400.0, 400.0, 300.0, 310.0, 350.0]
Market clearing price: 13.32
Min 13.32 ProductionGen[1] + 13.32 ProductionGen[2] + 20.7 ProductionGen[3] + 20.93 ProductionGen[4] + 26.11 ProductionGen[5] + 10.52 ProductionGen[6] + 10.52 ProductionGen[7] + 6.02 ProductionGen[8] + 5.47 ProductionGen[9] + 10.52 ProductionGen[11] + 10.89 ProductionGen[12]
Subject to
 BalanceConstraint : ProductionGen[1] + ProductionGen[2] + ProductionGen[3] + ProductionGen[4] + ProductionGen[5] + ProductionGen[6] + ProductionGen[7] + ProductionGen[8] + ProductionGen[9] + ProductionGen[10] + ProductionGen[11] + ProductionGen[12] == 2411.955
 ProductionGen[1] >= 30.4
 ProductionGen[2] >= 30.4
 ProductionGen[3]

Optimal solution found.
Total generation: 21796.697949999998
Generation per generator: [30.4, 30.4, 75.0, 206.85, 12.0, 155.0, 155.0, 400.0, 400.0, 300.0, 310.0, 337.30499999999984]
Market clearing price: 10.89
Min 13.32 ProductionGen[1] + 13.32 ProductionGen[2] + 20.7 ProductionGen[3] + 20.93 ProductionGen[4] + 26.11 ProductionGen[5] + 10.52 ProductionGen[6] + 10.52 ProductionGen[7] + 6.02 ProductionGen[8] + 5.47 ProductionGen[9] + 10.52 ProductionGen[11] + 10.89 ProductionGen[12]
Subject to
 BalanceConstraint : ProductionGen[1] + ProductionGen[2] + ProductionGen[3] + ProductionGen[4] + ProductionGen[5] + ProductionGen[6] + ProductionGen[7] + ProductionGen[8] + ProductionGen[9] + ProductionGen[10] + ProductionGen[11] + ProductionGen[12] == 2199.915
 ProductionGen[1] >= 30.4
 ProductionGen[2] >= 30.4
 ProductionGen[3] >= 75
 ProductionGen[4] >= 206.85
 ProductionGen[5] >= 12
 ProductionGen[6] >= 54.25
 ProductionGen[7] >= 54.25
 ProductionGen[8] >= 100
 ProductionGen[9] >= 100
 Product

ProductionGen[10] <= 300
 ProductionGen[11] <= 310
 ProductionGen[12] <= 350
Optimal solution found.
Total generation: 19493.034299999996
Generation per generator: [30.4, 30.4, 75.0, 206.85, 12.0, 155.0, 155.0, 400.0, 400.0, 300.0, 295.2649999999999, 140.0]
Market clearing price: 10.52
Min 13.32 ProductionGen[1] + 13.32 ProductionGen[2] + 20.7 ProductionGen[3] + 20.93 ProductionGen[4] + 26.11 ProductionGen[5] + 10.52 ProductionGen[6] + 10.52 ProductionGen[7] + 6.02 ProductionGen[8] + 5.47 ProductionGen[9] + 10.52 ProductionGen[11] + 10.89 ProductionGen[12]
Subject to
 BalanceConstraint : ProductionGen[1] + ProductionGen[2] + ProductionGen[3] + ProductionGen[4] + ProductionGen[5] + ProductionGen[6] + ProductionGen[7] + ProductionGen[8] + ProductionGen[9] + ProductionGen[10] + ProductionGen[11] + ProductionGen[12] == 1934.865
 ProductionGen[1] >= 30.4
 ProductionGen[2] >= 30.4
 ProductionGen[3] >= 75
 ProductionGen[4] >= 206.85
 ProductionGen[5] >= 12
 ProductionGen[6] >= 54.25
 Producti

ProductionGen[9] >= 100
 ProductionGen[10] >= 300
 ProductionGen[11] >= 108.5
 ProductionGen[12] >= 140
 ProductionGen[1] <= 152
 ProductionGen[2] <= 152
 ProductionGen[3] <= 350
 ProductionGen[4] <= 591
 ProductionGen[5] <= 60
 ProductionGen[6] <= 155
 ProductionGen[7] <= 155
 ProductionGen[8] <= 400
 ProductionGen[9] <= 400
 ProductionGen[10] <= 300
 ProductionGen[11] <= 310
 ProductionGen[12] <= 350
Optimal solution found.
Total generation: 16704.7083
Generation per generator: [30.4, 30.4, 75.0, 206.85, 12.0, 155.0, 76.71499999999992, 400.0, 400.0, 300.0, 108.5, 140.0]
Market clearing price: 10.52
Min 13.32 ProductionGen[1] + 13.32 ProductionGen[2] + 20.7 ProductionGen[3] + 20.93 ProductionGen[4] + 26.11 ProductionGen[5] + 10.52 ProductionGen[6] + 10.52 ProductionGen[7] + 6.02 ProductionGen[8] + 5.47 ProductionGen[9] + 10.52 ProductionGen[11] + 10.89 ProductionGen[12]
Subject to
 BalanceConstraint : ProductionGen[1] + ProductionGen[2] + ProductionGen[3] + ProductionGen[4] + Producti

14554.639799999999
Generation per generator: [30.4, 30.4, 75.0, 206.85, 12.0, 54.25, 54.25, 258.16499999999996, 400.0, 300.0, 108.5, 140.0]
Market clearing price: 6.02


In [56]:
MarketClearingResults_df

Row,hour,MarketClearingPrice,ProductionGen1,ProductionGen2,ProductionGen3,ProductionGen4,ProductionGen5,ProductionGen6,ProductionGen7,ProductionGen8,ProductionGen9,ProductionGen10,ProductionGen11,ProductionGen12
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,6.02,30.4,30.4,75.0,206.85,12.0,54.25,54.25,364.185,400.0,300.0,108.5,140.0
2,2,6.02,30.4,30.4,75.0,206.85,12.0,54.25,54.25,258.165,400.0,300.0,108.5,140.0
3,3,6.02,30.4,30.4,75.0,206.85,12.0,54.25,54.25,178.65,400.0,300.0,108.5,140.0
4,4,6.02,30.4,30.4,75.0,206.85,12.0,54.25,54.25,152.145,400.0,300.0,108.5,140.0
5,5,6.02,30.4,30.4,75.0,206.85,12.0,54.25,54.25,152.145,400.0,300.0,108.5,140.0
6,6,6.02,30.4,30.4,75.0,206.85,12.0,54.25,54.25,178.65,400.0,300.0,108.5,140.0
7,7,10.52,30.4,30.4,75.0,206.85,12.0,155.0,103.22,400.0,400.0,300.0,108.5,140.0
8,8,10.89,30.4,30.4,75.0,206.85,12.0,155.0,155.0,400.0,400.0,300.0,310.0,204.78
9,9,13.32,30.4,123.725,75.0,206.85,12.0,155.0,155.0,400.0,400.0,300.0,310.0,350.0


In [57]:
#save result_df to csv
CSV.write("../results/Task1market_clearing_Leon.csv", MarketClearingResults_df)

"../results/Task1market_clearing_Leon.csv"